In [ ]:
# ベイジアン非負値行列因子分解
# ライブラリを読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy
import scipy.stats as ss
from numpy.random import *
from scipy import optimize
from scipy.stats import norm

In [ ]:
# データの発生
# データの設定
hh = 3000   #ユーザー数
item = 500   #アイテム数
k = 10   #基底数

In [ ]:
# 非負値行列因子分解の仮定に基づきデータを生成
# ガンマ分布よりパラメータを生成
alpha01 = 0.2; beta01 = 1.0
alpha02 = 0.15; beta02 = 0.8
W0 = numpy.random.gamma(alpha01, int(1/beta01), hh*k).reshape(hh, k)
H0 = numpy.random.gamma(alpha02, int(1/beta02), item*k).reshape(k, item)
WH = np.dot(W0, H0)

In [ ]:
# ポアソン分布よりデータを生成
Data = np.zeros((hh, item))
for j in range(item):
    Data[:, j] = numpy.random.poisson(WH[:, j], hh)

# 購買数を確認
print(np.round(np.sum(Data, axis=0), 0))
print(np.round(np.sum(Data, axis=1), 0))

# ベストな対数尤度
LLbest = np.sum(scipy.stats.poisson.logpmf(Data, WH))
print(LLbest)

In [ ]:
# マルコフ連鎖モンテカルロ法で非負値行列因子分解を推定
# アルゴリズムの設定
R = 5000
keep = 2
disp = 100
burnin = int(1000/keep)

# 事前分布の設定
alpha1 = 0.01; beta1 = 0.01
alpha2 = 0.01; beta2 = 0.01

# 初期値の設定
W = numpy.random.gamma(0.1, 1/0.1, hh*k).reshape(hh, k)
H = numpy.random.gamma(0.1, 1/0.1, item*k).reshape(k, item)

# サンプリング結果の保存用配列
W_array = np.zeros((hh, k, int(R/keep)))
H_array = np.zeros((k, item, int(R/keep)))
LAMBDA = np.zeros((hh, item))

In [ ]:
# ギブスサンプリングでパラメータをサンプリング
for rp in range(R):
    
    # ガンマ分布からWをサンプリング
    WH = np.dot(W, H)
    Lambda = np.zeros((hh, item, k))
    # 補助変数lambdaを更新
    for j in range(k):
        Lambda[:, :, j] = np.dot(W[:, j].reshape(hh, 1), H[j, :].reshape(1, item)) / WH

    # ガンマ分布からパラメータを生成
    for j in range(k):
        w1 = alpha1 + np.sum(Lambda[:, :, j] * Data, axis=1)
        w2 = beta1 + sum(H[j, :])
        W[:, j] = numpy.random.gamma(w1, 1/w2, hh) 

    # 各列ベクトルの要素を正規化
    W = W / np.sum(W, axis=0).repeat(hh).reshape(k, hh).T * hh/5


    # ガンマ分布よりHをサンプリング
    WH = np.dot(W, H)
    Lambda = np.zeros((hh, item, k))
    # 補助変数lambdaを更新
    for j in range(k):
        Lambda[:, :, j] = np.dot(W[:, j].reshape(hh, 1), H[j, :].reshape(1, item)) / WH

    # ガンマ分布からパラメータを生成
    for j in range(k):
        h1 = alpha1 + np.sum(Lambda[:, :, j] * Data, axis=0)
        h2 = beta1 + sum(W[:, j])
        H[j, :] = numpy.random.gamma(h1, 1/h2, item) 


    # サンプリング結果の格納と表示
    if rp%keep==0:
        mkeep = int(rp/keep)
        W_array[:, :, mkeep] = W
        H_array[:, :, mkeep] = H

    if rp%disp==0:
        LL = np.sum(scipy.stats.poisson.logpmf(Data, WH))   # 非負値行列因子分解の対数尤度
        print(rp)
        print(np.round(np.array((LL, LLbest)), 1))